In [1]:
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage.filters import gabor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import cv2
import fnmatch
from skimage.feature import local_binary_pattern
from scipy.stats import itemfreq
from sklearn.preprocessing import normalize
import PIL
%matplotlib inline

In [2]:
imagePatches = glob('D:/breakhis/BreaKHis_v1/histology_slides/breast/**/SOB/**/**/40X/*.png', recursive=True)
pattern_adenosis = '*B_A*'
pattern_fibroadenoma = '*B_F*'
pattern_phyllodes_tumor = '*B_PT*'
pattern_tubular_adenoma = '*B_TA*'
pattern_ductal_carcinoma = '*M_DC*'
pattern_lobular_carcinoma = '*M_LC*'
pattern_mucinous_carcinoma = '*M_MC*'
pattern_papillary_carcinoma  = '*M_PC*'

adenosis = fnmatch.filter(imagePatches, pattern_adenosis)
fibroadenoma = fnmatch.filter(imagePatches, pattern_fibroadenoma)
phyllodes_tumor = fnmatch.filter(imagePatches, pattern_phyllodes_tumor)
tubular_adenoma = fnmatch.filter(imagePatches,pattern_tubular_adenoma)
ductal_carcinoma = fnmatch.filter(imagePatches, pattern_ductal_carcinoma)
lobular_carcinoma = fnmatch.filter(imagePatches, pattern_lobular_carcinoma)
mucinous_carcinoma = fnmatch.filter(imagePatches, pattern_mucinous_carcinoma)
papillary_carcinoma = fnmatch.filter(imagePatches, pattern_papillary_carcinoma)

X = []
y = []
for img in imagePatches:
    full_size_image = cv2.imread(img)
    im = cv2.resize(full_size_image, (299, 299), interpolation=cv2.INTER_CUBIC)
    img_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    img_arr = np.array(img_gray.reshape(299,299))
    
     # LBP
    feat_lbp = local_binary_pattern(img_arr,5,2,'uniform').reshape(299*299)
    lbp_hist,_ = np.histogram(feat_lbp,8)
    lbp_hist = np.array(lbp_hist,dtype=float)
    lbp_prob = np.divide(lbp_hist,np.sum(lbp_hist))
    lbp_energy = np.nansum(lbp_prob**2)
    lbp_entropy = -np.nansum(np.multiply(lbp_prob,np.log2(lbp_prob)))
    
    # GLCM
    gCoMat = greycomatrix(img_arr, [2], [0],256,symmetric=True, normed=True)
    contrast = greycoprops(gCoMat, prop='contrast')
    dissimilarity = greycoprops(gCoMat, prop='dissimilarity')
    homogeneity = greycoprops(gCoMat, prop='homogeneity')    
    energy = greycoprops(gCoMat, prop='energy')
    correlation = greycoprops(gCoMat, prop='correlation')    
    feat_glcm = np.array([contrast[0][0],dissimilarity[0][0],homogeneity[0][0],energy[0][0],correlation[0][0]])
    
    # Gabor filter
    gaborFilt_real,gaborFilt_imag = gabor(img_arr,frequency=0.6)
    gaborFilt = (gaborFilt_real**2+gaborFilt_imag**2)//2
    gabor_hist,_ = np.histogram(gaborFilt,8)
    gabor_hist = np.array(gabor_hist,dtype=float)
    gabor_prob = np.divide(gabor_hist,np.sum(gabor_hist))
    gabor_energy = np.nansum(gabor_prob**2)
    gabor_entropy = -np.nansum(np.multiply(gabor_prob,np.log2(gabor_prob)))
    
    # Concatenating features(2+5+2)    
    concat_feat = np.concatenate(([lbp_energy,lbp_entropy],feat_glcm,[gabor_energy,gabor_entropy]),axis=0)
    X.append(concat_feat)
    
    #creating target
    if img in adenosis:
        y.append(0)
    elif img in fibroadenoma:
        y.append(1)
    elif img in phyllodes_tumor:
        y.append(2)
    elif img in tubular_adenoma:
        y.append(3)
    elif img in ductal_carcinoma:
        y.append(4)
    elif img in lobular_carcinoma:
        y.append(5)
    elif img in mucinous_carcinoma:
        y.append(6)
    elif img in papillary_carcinoma:
        y.append(7)
    else:
        #break
        print('no class')

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log2
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply


In [3]:
X = np.array(X)
y = np.array(y)
np.save('C:/Users/Debanjan Bhadra/Desktop/project/x_BreakHis_40_manual_extracted_features.npy',X,allow_pickle=True)
np.save('C:/Users/Debanjan Bhadra/Desktop/project/y_BreakHis_40_bin.npy',y,allow_pickle=True)